In [1]:
from src.preprocessing import *
from src.master import *
from src.modelisation import *
from src.impute import *

pd.options.display.max_columns = None

In [2]:
df = master_preprocessing_X('data/Airport_Data.csv',
                      'data/Weather_data_prep.csv',
                      'data/ACchar.xlsx',
                      'Correspondance.pkl')

df['target'] = create_target(df)

df = impute_chocks_on(df)

df = df.dropna()

//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3325: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
variables = ['carrier','runway','stand','Manufacturer','acType','PRCP','TAVG','AWND','TMAX','TMIN','WDF2','WDF5','WSF2','WSF5','WT01','WT02','WT03','WT08','Approach Speed\n(Vref)']

encoded_df = encoding_df(df, variables)

In [4]:
encoded_df.head()

,PRCP,TAVG,AWND,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT03,WT08,Approach Speed (Vref),carrier_CAR1,carrier_CAR2,carrier_CAR3,carrier_CAR4,runway_RUNWAY01,runway_RUNWAY02,runway_RUNWAY03,runway_RUNWAY04,runway_RUNWAY05,runway_RUNWAY06,runway_RUNWAY07,runway_RUNWAY08,runway_RUNWAY09,runway_RUNWAY10,stand_STAND01,stand_STAND02,stand_STAND03,stand_STAND04,stand_STAND05,stand_STAND06,stand_STAND07,stand_STAND08,stand_STAND09,stand_STAND10,stand_STAND100,stand_STAND101,stand_STAND102,stand_STAND103,stand_STAND104,stand_STAND105,stand_STAND106,stand_STAND107,stand_STAND108,stand_STAND109,stand_STAND11,stand_STAND110,stand_STAND111,stand_STAND112,stand_STAND113,stand_STAND114,stand_STAND115,stand_STAND116,stand_STAND117,stand_STAND118,stand_STAND119,stand_STAND12,stand_STAND120,stand_STAND121,stand_STAND122,stand_STAND123,stand_STAND124,stand_STAND125,stand_STAND126,stand_STAND127,stand_STAND128,stand_STAND129,stand_STAND13,stand_STAND130,stand_STAND131,stand_STAND132,stand_STAND133,stand_STAND134,stand_STAND135,stand_STAND136,stand_STAND137,stand_STAND138,stand_STAND139,stand_STAND14,stand_STAND140,stand_STAND141,stand_STAND142,stand_STAND143,stand_STAND144,stand_STAND145,stand_STAND146,stand_STAND147,stand_STAND148,stand_STAND149,stand_STAND15,stand_STAND150,stand_STAND151,stand_STAND152,stand_STAND153,stand_STAND154,stand_STAND155,stand_STAND156,stand_STAND157,stand_STAND158,stand_STAND159,stand_STAND16,stand_STAND160,stand_STAND17,stand_STAND18,stand_STAND19,stand_STAND20,stand_STAND21,stand_STAND22,stand_STAND23,stand_STAND24,stand_STAND25,stand_STAND26,stand_STAND27,stand_STAND28,stand_STAND29,stand_STAND30,stand_STAND31,stand_STAND32,stand_STAND33,stand_STAND34,stand_STAND35,stand_STAND36,stand_STAND37,stand_STAND38,stand_STAND39,stand_STAND40,stand_STAND41,stand_STAND42,stand_STAND43,stand_STAND44,stand_STAND45,stand_STAND46,stand_STAND47,stand_STAND48,stand_STAND49,stand_STAND50,stand_STAND51,stand_STAND52,stand_STAND53,stand_STAND54,stand_STAND55,stand_STAND56,stand_STAND57,stand_STAND58,stand_STAND59,stand_STAND60,stand_STAND61,stand_STAND62,stand_STAND63,stand_STAND64,stand_STAND65,stand_STAND66,stand_STAND67,stand_STAND68,stand_STAND69,stand_STAND70,stand_STAND71,stand_STAND72,stand_STAND73,stand_STAND74,stand_STAND75,stand_STAND76,stand_STAND77,stand_STAND78,stand_STAND79,stand_STAND80,stand_STAND81,stand_STAND82,stand_STAND83,stand_STAND84,stand_STAND85,stand_STAND86,stand_STAND87,stand_STAND88,stand_STAND89,stand_STAND90,stand_STAND91,stand_STAND92,stand_STAND93,stand_STAND94,stand_STAND95,stand_STAND96,stand_STAND97,stand_STAND98,stand_STAND99,Manufacturer_Airbus,Manufacturer_Boeing,Manufacturer_Boeing (McDonnell Douglas),Manufacturer_Bombardier,acType_7hd,acType_a319,acType_a319-sl,acType_a320/2,acType_a321/2,acType_a330/2,acType_a330/3,acType_a350/9,acType_b717/2,acType_b737/7-wl,acType_b737/8,acType_b737/8-wl,acType_b737/9,acType_b737/9-wl,acType_b757/2-wl,acType_b757/3,acType_b757/3-wl,acType_b767/3,acType_b767/3-wl,acType_b777/2,acType_b777/2-lr,acType_crj/2,acType_crj/7,acType_crj/9,acType_cs100,acType_md88,acType_md90/3
0,0.06,79.0,4.173333,88.0,69.0,170.0,170.0,15.0,19.0,1.0,0.0,1.0,1.0,141.0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.06,79.0,4.173333,88.0,69.0,170.0,170.0,15.0,19.0,1.0,0.0,1.0,1.0,137.0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [5]:
from sklearn.model_selection import train_test_split
from sklearn import tree

# Define input 
X = encoding_df(df, variables)

# Set validation
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

clf = tree.DecisionTreeRegressor()
clf = clf.fit(X_train, y_train)

print(compute_mape(y_test, clf.predict(X_test)))
print(compute_rmse(y_test, clf.predict(X_test)))

y_pred = clf.predict(X_test)

0.59320365934935
6.036180938471881


In [6]:
# Define input 
X = encoding_df(df, variables)

# Set validation
y = df['target']

In [11]:
def master_modelisation(X,y,test_size,model_function_list) :
    """
    function aggregating models and building comparison table

    input: how many models, training features, target values,
    output : predicted values and rmse

    """    
    models = model_function_list
    
    output = pd.DataFrame()
    model = []
    RMSE = []
    MAPE = []
    tenth_perc = []    
    ninetieth_perc = []
    
    for i in models:
        output_row = pd.DataFrame()
        
        model.append(i)
        
        y_pred, y_test = i(df, variables, test_size)

        RMSE.append(compute_rmse(y_test, y_pred))
        MAPE.append(compute_mape(y_test, y_pred))
        tenth_perc.append(tenth_percentile(y_test, y_pred))   
        ninetieth_perc.append(ninetieth_percentile(y_test, y_pred))
        
    output['Model'] = model
    output['RMSE'] = RMSE
    output['MAPE'] = MAPE
    output['tenth_perc'] = tenth_perc
    output['ninetieth_perc'] = ninetieth_perc
        
    return output

# TEST
master_modelisation(X,y,0.10,[linear_reg,decision_tree])

Linear Regression RMSE 4.276731056435
6.04887294293215


,Model,RMSE,MAPE,tenth_perc,ninetieth_perc
0,<function linear_reg at 0x111336158>,4.276731,0.473814,20.068462,262.629185
1,<function decision_tree at 0x1113360d0>,6.048873,0.591438,0.000000,420.000000


In [ ]:
linear_reg(df, variables, 0.10)

In [ ]:
master_modelisation(models,X,y,test_size)